In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from functools import reduce
from joblib import Parallel, delayed

<IPython.core.display.Javascript object>

## Parameter Setting을 정의

In [3]:
save_path = Path("result")
data_path = Path("Data")

<IPython.core.display.Javascript object>

In [4]:
settings = {
    "seed": 1,
    "start_date": pd.Timestamp("1925-12-31"),
    # 'end_date': datetime.date(2022, 12, 31),  # end_date는 반드시 Last_CRSP_UPDATE 이하이어야 합니다
    "end_date": pd.Timestamp(
        "2022-12-31"
    ),  # end_date는 반드시 Last_CRSP_UPDATE 이하이어야 합니다
    "country_excl": ["ZWE", "VEN"],  # 데이터 문제로 인해 제외된 국가들
    "weighting": {  # 사용할 가중치 방식 (선택지: "ew", "vw", "vw_cap")
        "us": "vw_cap",
        "global_ex_us": "vw_cap",
    },
    "n_stocks_min": 5,  # 각 포트폴리오 측면에서 최소한의 주식 수
    "months_min": 5 * 12,  # 팩터가 포함되기 위해 필요한 최소한의 관측 기간
    "country_weighting": "market_cap",  # 국가 가중치를 부여하는 방법 ("market_cap", "stocks", "ew")
    "countries_min": 3,  # 지역 포트폴리오에 필요한 최소 국가 수
    "clusters": "hcl",  # 사용할 클러스터링 방법 (선택지: "manual", "hcl")
    "hcl": {
        "ret_type": "alpha",  # 클러스터링에 사용할 수익 유형 (선택지: "raw", "alpha")
        "cor_method": "pearson",  # 거리 계산에 사용할 상관관계 방법
        "linkage": "ward.D",  # 클러스터링에 사용할 연결 방법
        "k": 13,  # 색칠할 클러스터 수
        "region": "us",  # 클러스터링에 사용할 지역
        "start_year": 1975,  # 클러스터 데이터 시작 연도
    },
    "eb": {
        "scale_alpha": True,
        "overlapping": False,
        "min_obs": 5 * 12,
        "fix_alpha": True,
        "bs_cov": True,
        "shrinkage": 0,
        "cor_type": "block_clusters",
        "bs_samples": 10000,  # 부트스트랩 샘플 수 (논문에서는 10000으로 설정)
    },
    "tpf": {
        "start": {
            "world": pd.Timestamp("1952-1-1"),
            "us": pd.Timestamp("1952-1-1"),
            "developed": pd.Timestamp("1987-1-1"),
            "emerging": pd.Timestamp("1994-1-1"),
            "size_grps": pd.Timestamp("1963-1-1"),  # 나노캡 시작에 의해 결정됨
        },
        "bs_samples": 10000,  # 부트스트랩 샘플 수 [논문에서는 10,000]
        "shorting": False,  # 공매도가 허용되어야 하는가?
    },
    "tpf_factors": {
        "region": "us",
        "orig_sig": True,  # 원래 중요한 팩터만 포함: True, 모두 포함: (True, False)
        "start": pd.Timestamp("1972-1-31"),
        "scale": True,  # 사후 변동성 10%에 맞춰 조정할 것인가?
        "k": 5,  # 교차 검증을 위한 폴드 수
    },
}

<IPython.core.display.Javascript object>

# Market Return

**변수 설명**
- `excntry`: ISO Alpha-3 코드로 표시된 증권이 상장된 국가.
- `eom`: 수익률을 계산하는 데 사용된 월의 월말.
- `characteristic`: 특성의 이름, Jensen, Kelly, Pedersen (2022)의 표 J.1을 참조.
- `region`: 포함된 팩터들의 지역/MSCI 국가 개발 상태.
- `size_grp`: 순위 가중 팩터를 생성하는 데 사용된 사이즈 그룹.
- `pf`: 포트폴리오 식별자.
- `n`: 포트폴리오 내의 총 주식 수.
- `n_stocks`: 롱(long) 및 숏(short) 포트폴리오 내의 총 주식 수.
- `n_stocks_min`: 롱 및 숏 포트폴리오 내 최소 주식 수. 예를 들어, 롱 포트폴리오에 10개의 주식이 있고 숏 포트폴리오에 40개의 주식이 있으면, `n_stocks=50`이고 `n_stocks_min=10`.
- `n_countries`: 지역 포트폴리오에 포함된 국가 수.
- `signal` (pfs.csv): 포트폴리오 내 중간 특성 값.
- `signal` (hml.csv, lms.csv): 롱 및 숏 포트폴리오 내 중간 특성의 차이.
- `signal_weighted`: 순위 가중 신호.
- `ret_ew`: 동등 가중 수익률.
- `ret_vw`: 가치 가중 수익률.
- `ret_vw_cap`: Jensen, Kelly, Pedersen (2022)에서 사용된 제한된 가치 가중 수익률.
- `ret_weighted`: 순위 가중 수익률.
- `me_lag1`: 월 초 기준으로 한 국가 내 총 시장 자본 규모.
- `dolvol_lag1`: 전월에 국가 내에서 거래된 총 거래 금액.
- `stocks`: 시장 포트폴리오에 포함된 주식.
- `mkt_vw_lcl`: 현지 통화로 표시된 가치 가중 시장 수익률.
- `mkt_ew_lcl`: 현지 통화로 표시된 동등 가중 시장 수익률.
- `mkt_vw`: USD로 표시된 가치 가중 시장 수익률.
- `mkt_ew`: USD로 표시된 동등 가중 시장 수익률.
- `mkt_ew_exc`: USD로 표시된 동등 가중 초과 시장 수익률.
- `mkt_vw_exc` (market_return.csv): USD로 표시된 가치 가중 초과 시장 수익률.
- `mkt_vw_exc` (Regional Factors): 지역 포트폴리오에 포함된 국가들의 USD로 표시된 가치 가중 평균 초과 시장 수익률.

In [5]:
market_returns = pd.read_csv(data_path / "market_returns.csv")

<IPython.core.display.Javascript object>

In [6]:
market_returns["eom"] = pd.to_datetime(market_returns["eom"])

<IPython.core.display.Javascript object>

**예상대로 market return은 Long-Only Market Return이다.**

In [7]:
test_return = (
    market_returns[market_returns["excntry"] == "USA"]
    .loc[:, ["eom", "mkt_vw"]]
    .set_index("eom")
)
test_return.add(1).cumprod().plot(logy=True)

In [ ]:
market_returns = market_returns[
    (market_returns["eom"] >= settings["start_date"])
    & (market_returns["eom"] <= settings["end_date"])
    &
    # 데이터 문제로 제외할 국가들
    (~market_returns["excntry"].isin(settings["country_excl"]))
    & ~(
        (market_returns["excntry"] == "PER")
        & (market_returns["eom"] == pd.Timestamp("1992-01-31"))
        & (market_returns["mkt_vw_exc"] >= 8900)
    )
    & ~(
        (market_returns["excntry"] == "VEN")
        & (market_returns["eom"] == pd.Timestamp("2018-02-28"))
        & (market_returns["mkt_vw_exc"] < -1)
    )
]

<IPython.core.display.Javascript object>

## Labels

In [ ]:
char_info = pd.read_excel(
    "Factor Details.xlsx", sheet_name="details", usecols="A:N"
).dropna(subset=["abr_jkp"])

<IPython.core.display.Javascript object>

In [ ]:
char_info = char_info[["abr_jkp", "direction", "in-sample period", "group"]].rename(
    columns={
        "abr_jkp": "characteristic",
        "in-sample period": "date_range",
        "group": "hxz_group",
    }
)
char_info["direction"] = char_info["direction"].astype(int)
char_info["sample_start"] = char_info["date_range"].str.extract(r"(^\d+)").astype(int)
char_info["sample_end"] = char_info["date_range"].str.extract(r"(\d+$)").astype(int)
# 각종 요인들 정의
base_chars = char_info["characteristic"].tolist()

<IPython.core.display.Javascript object>

# Country Classification

In [ ]:
country_classification = pd.read_excel(
    "Country Classification.xlsx", sheet_name="countries", usecols="A:C"
).dropna(subset=["excntry"])

<IPython.core.display.Javascript object>

In [ ]:
country_classification.head()

,excntry,msci_development,region
0,USA,developed,north america
1,CHN,emerging,asia
2,JPN,developed,asia
3,HKG,developed,asia
4,GBR,developed,europe


<IPython.core.display.Javascript object>

## Regions

In [ ]:
region_info = pd.DataFrame(
    {
        "name": ["us", "developed", "emerging", "frontier", "world", "world_ex_us"],
        "country_codes": [
            ["USA"],
            country_classification[
                country_classification["msci_development"] == "developed"
            ]["excntry"].tolist(),
            country_classification[
                country_classification["msci_development"] == "emerging"
            ]["excntry"].tolist(),
            country_classification[
                country_classification["msci_development"] == "frontier"
            ]["excntry"].tolist(),
            country_classification["excntry"].tolist(),
            country_classification[country_classification["excntry"] != "USA"][
                "excntry"
            ].tolist(),
        ],
        "countries_min": [1] + [settings["countries_min"]] * 3 + [1, 3],
    }
)

<IPython.core.display.Javascript object>

In [ ]:
region_info = region_info.set_index("name").to_dict("index")

<IPython.core.display.Javascript object>

## HML

In [ ]:
hml = pd.read_csv(data_path / "hml.csv")
hml["eom"] = pd.to_datetime(hml["eom"], format="%Y-%m-%d")

<IPython.core.display.Javascript object>

### Choose weighting

In [ ]:
# 나라별 portfolio weighting method 정한 것을 ret column으로 사용
hml_usa = hml[hml["excntry"] == "USA"].copy()
hml_usa.loc[:, "ret"] = hml_usa.loc[:, [f"ret_{settings['weighting']['us']}"]]

hml_ex_usa = hml[hml["excntry"] != "USA"].copy()
hml_ex_usa.loc[:, "ret"] = hml_ex_usa.loc[
    :, [f"ret_{settings['weighting']['global_ex_us']}"]
]

hml = pd.concat([hml_usa, hml_ex_usa])

<IPython.core.display.Javascript object>

In [ ]:
hml.head()

,excntry,characteristic,eom,signal,n_stocks,n_stocks_min,ret_ew,ret_vw,ret_vw_cap,ret
2050119,USA,age,1926-03-31,1.0,502,8,0.066167,0.098837,0.086738,0.086738
2050120,USA,age,1926-04-30,1.0,505,12,-0.010546,0.006303,0.000572,0.000572
2050121,USA,age,1926-05-31,2.0,510,17,0.020290,0.000190,0.006332,0.006332
2050122,USA,age,1926-06-30,2.0,512,21,0.008927,0.008323,0.003012,0.003012
2050123,USA,age,1926-07-31,2.0,503,23,-0.017724,0.004220,-0.003211,-0.003211


<IPython.core.display.Javascript object>

## Filter

In [ ]:
hml = hml[
    (hml["characteristic"].isin(base_chars))
    & (hml["eom"] >= settings["start_date"])
    & (hml["eom"] <= settings["end_date"])
    & (~hml["ret"].isna())
    & (hml["n_stocks_min"] >= settings["n_stocks_min"])
    & (~hml["excntry"].isin(settings["country_excl"]))
]

<IPython.core.display.Javascript object>

In [ ]:
hml = hml.drop(columns=["signal", "n_stocks_min"])

<IPython.core.display.Javascript object>

In [ ]:
hml.head()

,excntry,characteristic,eom,n_stocks,ret_ew,ret_vw,ret_vw_cap,ret
2050119,USA,age,1926-03-31,502,0.066167,0.098837,0.086738,0.086738
2050120,USA,age,1926-04-30,505,-0.010546,0.006303,0.000572,0.000572
2050121,USA,age,1926-05-31,510,0.020290,0.000190,0.006332,0.006332
2050122,USA,age,1926-06-30,512,0.008927,0.008323,0.003012,0.003012
2050123,USA,age,1926-07-31,503,-0.017724,0.004220,-0.003211,-0.003211


<IPython.core.display.Javascript object>

### Set direction as original study

In [ ]:
hml = hml.merge(
    char_info[["characteristic", "direction"]], on="characteristic", how="left"
)
hml["ret"] = hml["ret"] * hml["direction"]
hml = hml.drop(columns=["ret_vw", "ret_ew", "ret_vw_cap"])

<IPython.core.display.Javascript object>

데이터 중복 check

In [ ]:
if hml.duplicated(subset=["characteristic", "excntry", "eom"]).any():
    raise Exception("HML is duplicated")

<IPython.core.display.Javascript object>

## Regional Portfolios

In [ ]:
def regional_data(data, market_returns, countries, weighting, countries_min, months_min, size_grps=False):
    __market_returns = market_returns.copy()
    __data = data.copy()

    __weights = __market_returns[__market_returns["excntry"].isin(countries)].copy()
    # 총 시장 자본 규모로 정의
    __weights["country_weight"] = __weights.apply(
        lambda x: (
            x["me_lag1"]
            if weighting == "market_cap"
            else (x["stocks"] if weighting == "stocks" else 1)
        ),
        axis=1,
    )
    __pf = __data[__data["excntry"].isin(countries)].merge(
        __weights[["excntry", "eom", "country_weight", "mkt_vw_exc"]],
        on=["excntry", "eom"],
        how="left",
    )

    if size_grps:
        __pf = __pf.groupby(['characteristic', 'size_grp', 'eom']).agg(
            n=('ret', 'size'),
            ret=('ret', lambda x: np.average(x, weights=__pf.loc[x.index, 'country_weight'])),
            mkt_vw_exc=('mkt_vw_exc', lambda x: np.average(x, weights=__pf.loc[x.index, 'country_weight']))
        ).reset_index()
    else:
        __pf = (
            __pf.groupby(["characteristic", "eom"])
            .agg(
                n=("ret", "size"),
                # Country Return을 Country weight로 결합해서 선형 결합함
                ret=(
                    "ret",
                    lambda x: np.average(x, weights=__pf.loc[x.index, "country_weight"]),
                ),
                # USD 가충초과 수익률을 Return을 Country weight로 결합해서 선형 결합함
                mkt_vw_exc=(
                    "mkt_vw_exc",
                    lambda x: np.average(x, weights=__pf.loc[x.index, "country_weight"]),
                ),
            )
            .reset_index()
        )



    # 최소 국가 처리: Countries
    __pf = __pf[__pf['n'] >= countries_min]

    # Factor 수익률 기간 필터링
    __pf['months'] = __pf.groupby('characteristic')['eom'].transform('size')
    __pf = __pf[__pf['months'] >= months_min]
    return __pf.copy()    

<IPython.core.display.Javascript object>

In [ ]:
iter_list = []
for key in tqdm(list(region_info.keys())):
    countries = region_info[key]["country_codes"]
    weighting = settings["country_weighting"]
    countries_min = region_info[key]["countries_min"]
    months_min = settings["months_min"]
    data = hml.copy()
    iter_list.append(
        {
            "data": hml.copy(),
            "market_returns": market_returns.copy(), 
            "countries": countries, 
            "weighting": weighting, 
            "countries_min": countries_min, 
            "months_min": months_min, 
        }
    )

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:00<00:00, 12.01it/s]


<IPython.core.display.Javascript object>

In [ ]:
parallel = Parallel(n_jobs=8, verbose=True)

<IPython.core.display.Javascript object>

In [ ]:
region_pf_list = parallel(delayed(regional_data)(**i) for i in iter_list)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of   6 | elapsed:   26.7s finished


<IPython.core.display.Javascript object>

In [ ]:
regional_pfs = pd.concat(region_pf_list)

<IPython.core.display.Javascript object>

In [ ]:
regional_pfs.tail()

,characteristic,eom,n,ret,mkt_vw_exc,months
68826,zero_trades_252d,2020-08-31,51,-0.002759,0.045821,333
68827,zero_trades_252d,2020-09-30,51,0.011594,-0.025302,333
68828,zero_trades_252d,2020-10-31,51,0.010563,-0.011245,333
68829,zero_trades_252d,2020-11-30,51,-0.036397,0.112102,333
68830,zero_trades_252d,2020-12-31,51,-0.005676,0.051994,333


<IPython.core.display.Javascript object>

In [ ]:
regional_pfs.to_parquet(save_path / "regional_pfs.parquet")

<IPython.core.display.Javascript object>

## Characteristic Managed Portfolios

In [ ]:
cmp = pd.read_csv(data_path / "cmp.csv")
cmp["eom"] = pd.to_datetime(cmp["eom"])

<IPython.core.display.Javascript object>

In [ ]:
cmp = cmp.rename(columns={'ret_weighted': 'ret'})
cmp = cmp[
    (cmp['characteristic'].isin(base_chars)) &
    (cmp['eom'] >= settings['start_date']) &
    (cmp['eom'] <= settings['end_date']) &
    (~cmp['ret'].isna()) &
    (cmp['signal_weighted'] != 0) &
    (cmp['n_stocks'] >= settings['n_stocks_min'] * 2) &
    (~cmp['excntry'].isin(settings['country_excl']))
]
cmp = cmp.drop(columns=['signal_weighted', 'n_stocks'])

<IPython.core.display.Javascript object>

In [ ]:
cmp = cmp.merge(char_info[['characteristic', 'direction']], on='characteristic', how='left')
cmp['ret'] = cmp['ret'] * cmp['direction']

<IPython.core.display.Javascript object>

In [ ]:
if cmp.duplicated(subset=['characteristic', 'excntry', 'size_grp', 'eom']).any():
    raise Warning("THE DATA HAS DUPLICATES")

<IPython.core.display.Javascript object>

In [ ]:
iter_list = []
for key in tqdm(list(region_info.keys())):
    countries = region_info[key]["country_codes"]
    weighting = settings["country_weighting"]
    countries_min = region_info[key]["countries_min"]
    months_min = settings["months_min"]
    data = hml.copy()
    iter_list.append(
        {
            "data": cmp.copy(),
            "market_returns": market_returns.copy(), 
            "countries": countries, 
            "weighting": weighting, 
            "countries_min": countries_min, 
            "months_min": months_min, 
            "size_grps": True, 
        }
    )

100%|██████████| 6/6 [00:00<00:00, 21.23it/s]


<IPython.core.display.Javascript object>

In [ ]:
regional_pfs_cmp_list = parallel(delayed(regional_data)(**i) for i in iter_list)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   6 out of   6 | elapsed:  1.8min finished


<IPython.core.display.Javascript object>

In [ ]:
regional_pfs_cmp = pd.concat(regional_pfs_cmp_list)

<IPython.core.display.Javascript object>

In [ ]:
regional_pfs_cmp.tail()

,characteristic,size_grp,eom,n,ret,mkt_vw_exc,months
656669,zero_trades_252d,small,2020-08-31,1,-0.004566,0.074068,5377
656670,zero_trades_252d,small,2020-09-30,1,-0.002624,-0.036228,5377
656671,zero_trades_252d,small,2020-10-31,1,0.025459,-0.020973,5377
656672,zero_trades_252d,small,2020-11-30,1,-0.130096,0.128030,5377
656673,zero_trades_252d,small,2020-12-31,1,-0.005767,0.046276,5377


<IPython.core.display.Javascript object>

In [ ]:
regional_pfs_cmp.to_parquet(save_path / "regional_pfs_cmp.parquet")

<IPython.core.display.Javascript object>

## Regional Market Returns

In [ ]:
regional_mkt_ret_list = []
for key in tqdm(list(region_info.keys())):
    countries = region_info[key]["country_codes"]
    weighting = settings["country_weighting"]
    countries_min = region_info[key]["countries_min"]
    months_min = settings["months_min"]

    __market_returns = market_returns.copy() 
    __pf = __market_returns[__market_returns["excntry"].isin(countries)].groupby('eom').agg(
        n=('mkt_vw_exc', 'size'),
        market=('mkt_vw_exc', lambda x: np.average(x, weights=market_returns.loc[x.index, 'me_lag1']))
    )
    __pf = __pf[__pf["n"] >= countries_min] 
    __pf = __pf.assign(regime=key) 
    regional_mkt_ret_list.append(__pf)             

100%|██████████| 6/6 [00:00<00:00, 17.46it/s]


<IPython.core.display.Javascript object>

In [ ]:
regional_mkt_ret = pd.concat(regional_mkt_ret_list)

<IPython.core.display.Javascript object>

In [ ]:
regional_mkt_ret.to_parquet(save_path / "regional_mkt_ret.parquet")

<IPython.core.display.Javascript object>